In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import tensorflow_addons as tfa
import glob
import wandb
from wandb.keras import WandbCallback
import shutil
import matplotlib.pyplot as plt
import numpy as np

/home/lorenzoquerol/miniconda3/envs/wssv-recognition/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
EXPERIMENT_NAME = "mobilenetv3small-average-runs"
DATA_DIR = "dataset-augmented"
AUTOTUNE = tf.data.AUTOTUNE
CONFIG = dict(
    epochs=100,
    learning_rate=1e-4,
    batch_size=32,
    img_shape=(224, 224),
    input_shape=(224, 224, 3),
    num_classes=2,
    dropout_rate=0.5,
    es_patience=10,
    seed_value=42,
)

In [3]:
def train():
    image_count = len(glob.glob(f"{DATA_DIR}/*/*/*.jpg"))
    healthy_count = len(glob.glob(f"{DATA_DIR}/train/healthy/*.jpg"))
    wssv_count = len(glob.glob(f"{DATA_DIR}/train/wssv/*.jpg"))

    train_dir = os.path.join(DATA_DIR, "train")
    valid_dir = os.path.join(DATA_DIR, "val")
    test_dir = os.path.join(DATA_DIR, "test")

    train_set = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        seed=CONFIG["seed_value"],
        image_size=CONFIG["img_shape"],
        batch_size=CONFIG["batch_size"],
        label_mode="categorical",
    ).prefetch(buffer_size=AUTOTUNE)

    valid_set = tf.keras.utils.image_dataset_from_directory(
        valid_dir,
        seed=CONFIG["seed_value"],
        image_size=CONFIG["img_shape"],
        batch_size=CONFIG["batch_size"],
        label_mode="categorical",
    ).prefetch(buffer_size=AUTOTUNE)

    test_set = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        seed=CONFIG["seed_value"],
        image_size=CONFIG["img_shape"],
        batch_size=CONFIG["batch_size"],
        label_mode="categorical",
    ).prefetch(buffer_size=AUTOTUNE)

    base_model = tf.keras.applications.MobileNetV3Small(
        input_shape=CONFIG["input_shape"],
        include_top=False,
        classes=CONFIG["num_classes"],
    )
    base_model.trainable = False

    inputs = tf.keras.Input(shape=CONFIG["input_shape"])
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(rate=CONFIG["dropout_rate"], seed=CONFIG["seed_value"])(
        x
    )
    outputs = tf.keras.layers.Dense(units=CONFIG["num_classes"], activation="softmax")(
        x
    )

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=CONFIG["learning_rate"],
        ),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
        metrics=[
            tf.keras.metrics.Precision(name="precision"),
            tf.keras.metrics.Recall(name="recall"),
            tfa.metrics.F1Score(
                num_classes=CONFIG["num_classes"],
                average="weighted",
                name="f1_score",
                threshold=0.5,
            ),
            tf.keras.metrics.FalseNegatives(name="false_negatives"),
            tf.keras.metrics.TruePositives(name="true_positives"),
            tf.keras.metrics.FalsePositives(name="false_positives"),
            tf.keras.metrics.TrueNegatives(name="true_negatives"),
        ],
    )

    class_0_weight = (1 / healthy_count) * (image_count / 2.0)
    class_1_weight = (1 / wssv_count) * (image_count / 2.0)
    class_weight = {0: class_0_weight, 1: class_1_weight}

    if os.path.exists("checkpoints_mobilenetv3small"):
        shutil.rmtree("checkpoints_mobilenetv3small")
        os.makedirs("checkpoints_mobilenetv3small")

    checkpoint_path = "checkpoints_mobilenetv3small/cp-{epoch:04d}.ckpt"

    model.save_weights(checkpoint_path.format(epoch=0))

    history = model.fit(
        train_set,
        epochs=CONFIG["epochs"],
        validation_data=valid_set,
        class_weight=class_weight,
        callbacks=[
            tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_path,
                save_weights_only=True,
                monitor="val_loss",
                mode="min",
                verbose=1,
                save_best_only=True,
            ),
            WandbCallback(save_model=False),
        ],
    )

    return history

In [4]:
for i in range(5):
    wandb.init(
        project="wssv-recognition",
        config=CONFIG,
        group=EXPERIMENT_NAME,
        job_type="train",
    )
    train()
    wandb.finish()

wandb: Currently logged in as: renzo_querol. Use `wandb login --relogin` to force relogin


Found 1023 files belonging to 2 classes.
Found 49 files belonging to 2 classes.
Found 13 files belonging to 2 classes.
Epoch 1/100
32/32 [==============================] - ETA: 0s - loss: 1.0024 - precision: 0.5630 - recall: 0.5630 - f1_score: 0.5917 - false_negatives: 447.0000 - true_positives: 576.0000 - false_positives: 447.0000 - true_negatives: 576.0000
Epoch 1: val_loss improved from inf to 0.65036, saving model to checkpoints_mobilenetv3small/cp-0001.ckpt
32/32 [==============================] - 8s 67ms/step - loss: 1.0024 - precision: 0.5630 - recall: 0.5630 - f1_score: 0.5917 - false_negatives: 447.0000 - true_positives: 576.0000 - false_positives: 447.0000 - true_negatives: 576.0000 - val_loss: 0.6504 - val_precision: 0.7143 - val_recall: 0.7143 - val_f1_score: 0.7051 - val_false_negatives: 14.0000 - val_true_positives: 35.0000 - val_false_positives: 14.0000 - val_true_negatives: 35.0000
Epoch 2/100
32/32 [==============================] - ETA: 0s - loss: 0.9381 - precision: 

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▁▂▂▃▃▄▄▄▄▅▅▆▆▅▆▆▆▆▆▇▆▇▇▇▇▇██▇▇██████████
false_negatives,█▇▇▇▆▅▅▅▄▄▄▃▃▄▃▃▃▃▃▂▃▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁
false_positives,█▇▇▇▆▅▅▅▄▄▄▃▃▄▃▃▃▃▃▂▃▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁
loss,█▇▆▆▅▅▄▄▄▄▃▃▃▃▂▂▃▂▂▂▂▂▂▁▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁
precision,▁▂▂▂▃▄▄▄▅▅▅▆▆▅▆▆▆▆▆▇▆▇▇▇▇▇██▇▇██████████
recall,▁▂▂▂▃▄▄▄▅▅▅▆▆▅▆▆▆▆▆▇▆▇▇▇▇▇██▇▇██████████
true_negatives,▁▂▂▂▃▄▄▄▅▅▅▆▆▅▆▆▆▆▆▇▆▇▇▇▇▇██▇▇██████████
true_positives,▁▂▂▂▃▄▄▄▅▅▅▆▆▅▆▆▆▆▆▇▆▇▇▇▇▇██▇▇██████████
val_f1_score,▁▄▂▃▅▅▅▅▅▅▆▆▅█▅▇▇▇▇▇▇▇▇▆▇▆▆▇▆▆▆▆▆▆▆▆▆▆▆▆
val_false_negatives,█▅█▆▄▄▅▄▅▅▃▃▄▁▄▂▂▂▂▂▂▂▂▃▂▃▃▂▃▃▃▃▃▄▃▃▃▃▃▃


Found 1023 files belonging to 2 classes.
Found 49 files belonging to 2 classes.
Found 13 files belonging to 2 classes.
Epoch 1/100
31/32 [============================>.] - ETA: 0s - loss: 1.0389 - precision: 0.5968 - recall: 0.5968 - f1_score: 0.6150 - false_negatives: 400.0000 - true_positives: 592.0000 - false_positives: 400.0000 - true_negatives: 592.0000
Epoch 1: val_loss improved from inf to 0.49712, saving model to checkpoints_mobilenetv3small/cp-0001.ckpt
32/32 [==============================] - 5s 62ms/step - loss: 1.0415 - precision: 0.5943 - recall: 0.5943 - f1_score: 0.6128 - false_negatives: 415.0000 - true_positives: 608.0000 - false_positives: 415.0000 - true_negatives: 608.0000 - val_loss: 0.4971 - val_precision: 0.7959 - val_recall: 0.7959 - val_f1_score: 0.7959 - val_false_negatives: 10.0000 - val_true_positives: 39.0000 - val_false_positives: 10.0000 - val_true_negatives: 39.0000
Epoch 2/100
31/32 [============================>.] - ETA: 0s - loss: 0.9273 - precision: 

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▁▁▂▂▃▃▃▄▅▅▅▅▆▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇█▇██████████
false_negatives,▇█▇▇▆▆▆▅▄▄▄▄▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁
false_positives,▇█▇▇▆▆▆▅▄▄▄▄▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁
loss,█▇▆▆▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▂▁▁▁▁▁▁▁
precision,▂▁▂▂▃▃▃▄▅▅▅▅▆▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇█▇██████████
recall,▂▁▂▂▃▃▃▄▅▅▅▅▆▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇█▇██████████
true_negatives,▂▁▂▂▃▃▃▄▅▅▅▅▆▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇█▇██████████
true_positives,▂▁▂▂▃▃▃▄▅▅▅▅▆▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇█▇██████████
val_f1_score,▄▁▃▄▆▆▇▆▆▆▆▆▇▇▇██▇▇▇███████▇█████▇████▇█
val_false_negatives,▅█▆▅▃▃▂▃▃▃▃▃▂▂▂▁▁▂▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▂▁▁▁▁▂▁


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▂▁▁▂▃▃▃▄▄▅▅▅▆▅▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇████▇██████
false_negatives,▇██▇▆▇▆▅▅▄▄▄▃▄▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁
false_positives,▇██▇▆▇▆▅▅▄▄▄▃▄▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁
loss,█▇▆▅▅▅▄▄▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision,▂▁▁▂▃▂▃▄▄▅▅▅▆▅▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇████▇██████
recall,▂▁▁▂▃▂▃▄▄▅▅▅▆▅▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇████▇██████
true_negatives,▂▁▁▂▃▂▃▄▄▅▅▅▆▅▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇████▇██████
true_positives,▂▁▁▂▃▂▃▄▄▅▅▅▆▅▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇████▇██████
val_f1_score,▂▁▂▂▅▅▇▇▇▇▇▇▇▇▇▇█████▇██████████████████
val_false_negatives,▇█▇▇▅▄▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


Found 1023 files belonging to 2 classes.
Found 49 files belonging to 2 classes.
Found 13 files belonging to 2 classes.
Epoch 1/100
31/32 [============================>.] - ETA: 0s - loss: 1.3156 - precision: 0.6391 - recall: 0.6391 - f1_score: 0.6332 - false_negatives: 358.0000 - true_positives: 634.0000 - false_positives: 358.0000 - true_negatives: 634.0000
Epoch 1: val_loss improved from inf to 0.54665, saving model to checkpoints_mobilenetv3small/cp-0001.ckpt
32/32 [==============================] - 6s 65ms/step - loss: 1.3200 - precision: 0.6383 - recall: 0.6383 - f1_score: 0.6326 - false_negatives: 370.0000 - true_positives: 653.0000 - false_positives: 370.0000 - true_negatives: 653.0000 - val_loss: 0.5467 - val_precision: 0.7551 - val_recall: 0.7551 - val_f1_score: 0.7551 - val_false_negatives: 12.0000 - val_true_positives: 37.0000 - val_false_positives: 12.0000 - val_true_negatives: 37.0000
Epoch 2/100
32/32 [==============================] - ETA: 0s - loss: 1.1302 - precision: 

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▂▁▁▂▂▃▄▄▄▅▅▅▅▅▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇███▇█████
false_negatives,▆██▇▇▆▅▅▅▄▄▄▄▄▃▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁
false_positives,▆██▇▇▆▅▅▅▄▄▄▄▄▃▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁
loss,█▆▅▅▄▄▃▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
precision,▃▁▁▂▂▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▆▇▇▇▇▇▇▇▇▇▇███▇█████
recall,▃▁▁▂▂▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▆▇▇▇▇▇▇▇▇▇▇███▇█████
true_negatives,▃▁▁▂▂▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▆▇▇▇▇▇▇▇▇▇▇███▇█████
true_positives,▃▁▁▂▂▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▆▇▇▇▇▇▇▇▇▇▇███▇█████
val_f1_score,▄▁▁▃▄▄▅▄▆▆▆▆▇███▇██████████▇██████▇▇████
val_false_negatives,▅██▆▅▅▅▅▃▃▃▃▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▂▂▁▁▁▁


Found 1023 files belonging to 2 classes.
Found 49 files belonging to 2 classes.
Found 13 files belonging to 2 classes.
Epoch 1/100
31/32 [============================>.] - ETA: 0s - loss: 0.9935 - precision: 0.6442 - recall: 0.6442 - f1_score: 0.6540 - false_negatives: 353.0000 - true_positives: 639.0000 - false_positives: 353.0000 - true_negatives: 639.0000
Epoch 1: val_loss improved from inf to 0.68593, saving model to checkpoints_mobilenetv3small/cp-0001.ckpt
32/32 [==============================] - 5s 68ms/step - loss: 0.9936 - precision: 0.6442 - recall: 0.6442 - f1_score: 0.6539 - false_negatives: 364.0000 - true_positives: 659.0000 - false_positives: 364.0000 - true_negatives: 659.0000 - val_loss: 0.6859 - val_precision: 0.5918 - val_recall: 0.5918 - val_f1_score: 0.5918 - val_false_negatives: 20.0000 - val_true_positives: 29.0000 - val_false_positives: 20.0000 - val_true_negatives: 29.0000
Epoch 2/100
32/32 [==============================] - ETA: 0s - loss: 0.9294 - precision: 

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▁▁▁▂▃▃▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇█▇▇▇▇▇█▇█▇▇▇██████
false_negatives,▇██▇▆▆▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▁▂▂▂▂▂▁▂▁▂▂▂▁▁▁▁▁▁
false_positives,▇██▇▆▆▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▁▂▂▂▂▂▁▂▁▂▂▂▁▁▁▁▁▁
loss,█▇▆▅▅▅▄▄▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▁▁▁▁▁▁▁
precision,▂▁▁▂▃▃▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇█▇▇▇▇▇█▇█▇▇▇██████
recall,▂▁▁▂▃▃▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇█▇▇▇▇▇█▇█▇▇▇██████
true_negatives,▂▁▁▂▃▃▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇█▇▇▇▇▇█▇█▇▇▇██████
true_positives,▂▁▁▂▃▃▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇█▇▇▇▇▇█▇█▇▇▇██████
val_f1_score,▁▃▅▅▅▆▇▇▆▇██████████████████████████████
val_false_negatives,█▆▅▄▄▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
